<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/oddiya_googleSDK_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================================================
#  CELL 1: SETUP - INSTALL LIBRARIES
# =======================================================================
# We now use the official Google SDK instead of LangChain's wrapper.
!pip install -qU google-generativeai pydantic

print("✅ 라이브러리 설치 완료. (Google SDK 버전)")

✅ 라이브러리 설치 완료. (Google SDK 버전)


# =======================================================================
#  CELL 2: IMPORTS & API KEY CONFIGURATION
# =======================================================================
import os
import json
import google.generativeai as genai
from google.colab import userdata

# Pydantic is still used for data validation.
from pydantic import BaseModel, Field
from typing import List

# --- API Key Configuration ---
try:
    # Configure the Google SDK with the API key
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    print("✅ Google Generative AI SDK 설정 완료.")
except Exception as e:
    print(f"🚨 SECRET NOT FOUND 또는 다른 오류 발생: {e}. Colab Secrets에서 'GOOGLE_API_KEY'를 설정했는지 확인하세요.")

In [ ]:
# =======================================================================
#  CELL 3: BACKEND - DEFINE DATA STRUCTURES
# =======================================================================
# NOTE: These Pydantic models are the same. They are our data contract.

class GenerativeActivity(BaseModel):
    name: str = Field(description="The name of the activity. The value MUST be in Korean (한국어).")
    time: str = Field(description="The suggested time for the activity (e.g., '오전 9시 - 11시'). The value MUST be in Korean (한국어).")
    description: str = Field(description="A brief description of the activity. The value MUST be in Korean (한국어).")

class DailyGenerativePlan(BaseModel):
    day_number: int = Field(description="The day number for this plan (e.g., 1, 2).")
    city: str = Field(description="The city for this plan. The value for this key MUST be in Korean (한국어).")
    theme_of_the_day: str = Field(description="A fun theme for the day. The value for this key MUST be in Korean (한국어).")
    activities: List[GenerativeActivity] = Field(description="A list of activities planned for the day.")

class FullTripPlan(BaseModel):
    """The complete, structured set of generated recommendations for the trip."""
    plans: List[DailyGenerativePlan]

print("✅ 백엔드 데이터 구조 정의 완료.")

✅ 백엔드 데이터 구조 정의 완료.


In [ ]:
# =======================================================================
#  CELL 4: BACKEND - MODEL AND PROMPT DEFINITION
# =======================================================================
# We instantiate the model directly from the Google SDK.
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# We create the prompt manually. Note how we now describe the JSON structure in plain text,
# as we don't have LangChain's parser to do it for us.
def create_prompt(city: str, num_days: int) -> str:
    # This function creates the detailed prompt that the model will receive.
    return f"""
You are an expert travel planner for families. Your task is to generate a JSON object that represents a travel plan.

**Critical Rules:**
1. The entire output must be a single, valid JSON object.
2. The JSON **keys** (like "plans", "city", "activities", "name", "time", "description") MUST be in English.
3. The JSON **values** (the actual content like the theme, descriptions, and place names) MUST be written in Korean (한국어).

**JSON Schema to follow:**
{{
  "plans": [
    {{
      "day_number": integer,
      "city": string (must be in Korean),
      "theme_of_the_day": string (must be in Korean),
      "activities": [
        {{
          "name": string (must be in Korean),
          "time": string (e.g., "오전 9시 - 11시"),
          "description": string (must be in Korean)
        }}
      ]
    }}
  ]
}}

**Request:**
Please generate a complete {num_days}-day trip itinerary for the city of **{city}**.
"""

print("✅ Google SDK 모델 및 프롬프트 생성 함수 준비 완료.")

✅ Google SDK 모델 및 프롬프트 생성 함수 준비 완료.


In [ ]:
# =======================================================================
#  CELL 5: DEFINE INPUT & RUN THE ENGINE (Google SDK Version)
# =======================================================================

# --- 여기에 입력을 수정하세요 ---
trip_plan = {
    "파리": 2,
    "로마": 2
}
# --------------------------------

# This tells the Gemini API to directly output a JSON string.
generation_config = genai.GenerationConfig(response_mime_type="application/json")

print("===================================================")
print(" G.O.A.T. 생성형 엔진 (Google SDK 직접 호출) ")
print("===================================================")
print(f"\n여행 계획: {trip_plan}")

all_plans = []
full_day_counter = 1

# Iterate through the user-defined trip plan
for city, num_days in trip_plan.items():
    print(f"\n[엔진] {city}를 위한 {num_days}일 계획을 Gemini에게 요청 중...")

    try:
        # Create the specific prompt for this city and number of days
        prompt = create_prompt(city, num_days)

        # Call the model using the native SDK
        response = model.generate_content(prompt, generation_config=generation_config)

        # Manually parse the JSON string from the model's response
        response_json = json.loads(response.text)

        # Manually validate the data against our Pydantic model
        # This replaces LangChain's OutputParser
        result = FullTripPlan.model_validate(response_json)

        # Process and number the days sequentially for the whole trip
        for plan in result.plans:
            plan.day_number = full_day_counter
            all_plans.append(plan)
            full_day_counter += 1

    except Exception as e:
        print(f"[오류] {city} 추천 생성 중 오류 발생: {e}")
        # If there's an error, print the raw response for debugging
        if 'response' in locals():
            print(f"--- RAW MODEL OUTPUT ---\n{response.text}\n------------------------")


# --- 최종 결과를 콘솔에 출력 ---
print("\n\n===================================================")
print("          ✨ 최종 생성된 여행 계획 ✨")
print("===================================================")

if not all_plans:
    print("\n생성된 계획이 없습니다.")
else:
    for day_plan in all_plans:
        print("\n---------------------------------------------------")
        print(f"  {day_plan.day_number}일차: {day_plan.city}에서의 '{day_plan.theme_of_the_day}'")
        print("---------------------------------------------------")

        for activity in day_plan.activities:
            print(f"\n  🕒 {activity.time}: {activity.name}")
            print(f"  👉 {activity.description}")

print("\n\n===================================================")
print("           [엔진 실행 완료]")
print("===================================================")

 G.O.A.T. 생성형 엔진 (Google SDK 직접 호출) 

여행 계획: {'파리': 2, '로마': 2}

[엔진] 파리를 위한 2일 계획을 Gemini에게 요청 중...

[엔진] 로마를 위한 2일 계획을 Gemini에게 요청 중...


          ✨ 최종 생성된 여행 계획 ✨

---------------------------------------------------
  1일차: 파리에서의 '파리의 명소 탐험'
---------------------------------------------------

  🕒 오전 9시 - 12시: 에펠탑 방문
  👉 에펠탑에 올라 파리 시내의 아름다운 전경을 감상하고, 기념 사진을 찍습니다. 매표소에서 줄이 길 수 있으니 미리 예약하는 것이 좋습니다.

  🕒 오후 1시 - 4시: 루브르 박물관 관람
  👉 세계적으로 유명한 루브르 박물관에서 모나리자를 비롯한 다양한 예술 작품을 감상합니다.  관람 시간이 길기 때문에 관심 있는 작품 위주로 선택하여 관람하는 것이 좋습니다.  미리 예약하면 줄을 서지 않아도 됩니다.

  🕒 오후 5시 - 8시: 센 강변 산책 및 저녁 식사
  👉 센 강변을 따라 산책하며 아름다운 파리의 야경을 감상하고, 맛있는 저녁 식사를 합니다. 센 강변에는 다양한 레스토랑이 있으므로 취향에 맞는 곳을 선택할 수 있습니다.

---------------------------------------------------
  2일차: 파리에서의 '파리 문화 체험'
---------------------------------------------------

  🕒 오전 9시 - 12시: 몽마르트 언덕 방문 및 거리 화가 관람
  👉 몽마르트 언덕에 올라 파리의 전경을 감상하고, 거리 화가들의 작품을 감상합니다.  사크레쾨르 대성당도 방문할 수 있습니다.

  🕒 오후 1시 - 5시: 베르사유 궁전 방문
  👉 파리 근교에 위치한 베르사유 궁전을 방문하여 화려한 궁전과 정원을 둘러봅니다